# POI based analysis

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format

## read data
* od matrices: Prt, PuT

In [15]:
od_PuT = pd.read_csv('data//MTX_203.csv')
del od_PuT['Unnamed: 0']
od_PuT.columns = [int(c) for c in od_PuT.columns]
od_PrT = pd.read_csv('data//MTX_204.csv')
del od_PrT['Unnamed: 0']
od_PrT.columns = [int(c) for c in od_PrT.columns]

In [16]:
od_PrT = od_PrT.stack().reset_index()
od_PrT.columns=['Z_Rejon',"Do_Rejon", 'TripsPrT']
od_PuT = od_PuT.stack().reset_index()
od_PuT.columns=['Z_Rejon',"Do_Rejon", 'TripsPuT']
od = pd.merge(od_PrT, od_PuT, on = ['Z_Rejon', "Do_Rejon"])

* trips

In [17]:
trips = pd.read_csv('data//From_Via_To.csv')
del trips['Unnamed: 0']

,Z_Rejon,POI,Do_Rejon,Czas_PuT,Czas_PrT
0,1.00,1.00,1.00,"2,283.62",520.28
1,1.00,1.00,2.00,"2,221.62",654.82
2,1.00,1.00,3.00,"2,221.62",636.82
3,1.00,1.00,4.00,"2,504.62",667.21
4,1.00,1.00,5.00,"3,013.62",785.48


## prepare

In [34]:
trips.Czas_PrT = trips.Czas_PrT/60 # przeliczam na minuty
trips.Czas_PuT = trips.Czas_PuT/60 *0.5 # tu zakladam dwa razy krotsze, dla obliczen, w Warszawie tak nie bedzie
BUDGET_PRT = trips['Czas_PrT'].quantile(0.5) # przyjmuje ze liczy sie tylko dolna polowa czasow
BUDGET_PUT = BUDGET_PRT #trips['Czas_PuT'].quantile(0.5) # a tu juz zakladam ze budzety sa rowne
trips['time_left_PrT']=BUDGET_PRT - trips.Czas_PrT # ile czasu zostaje ponizej budzetu w POI
trips['time_left_PuT']=BUDGET_PUT - trips.Czas_PuT
pd.options.display.float_format = '{:,.1f}'.format

In [18]:
trips = pd.merge(trips,od, on = ['Z_Rejon',"Do_Rejon"])
trips['odPair'] = trips["Z_Rejon"].astype(int)*1000+trips["Do_Rejon"].astype(int)

,Z_Rejon,POI,Do_Rejon,Czas_PuT,Czas_PrT,TripsPrT,TripsPuT,odPair
0,1.00,1.00,1.00,"2,283.62",520.28,2.35,1.60,1001
1,1.00,2.00,1.00,"2,781.72",489.47,2.35,1.60,1001
2,1.00,3.00,1.00,"4,538.03","1,070.58",2.35,1.60,1001
3,1.00,4.00,1.00,"2,372.85",517.89,2.35,1.60,1001
4,1.00,5.00,1.00,"3,400.56",833.29,2.35,1.60,1001


# POI-based analysis:
## number of people

In [26]:
print('number of people (potentially) per POI - PuT/PrT')
POI_ppl = trips[trips.Czas_PrT<BUDGET_PRT].groupby(by=['POI'])['TripsPrT'].sum().to_frame('nPleoplePrT')
POI_ppl['nPeoplePuT'] = trips[trips.Czas_PuT<BUDGET_PUT].groupby(by=['POI'])['TripsPuT'].sum()
POI_ppl

number of people (potentially) per POI - PuT/PrT


,nPleoplePrT,nPeoplePuT
POI,,
1.0,"5,971.2",60.2
2.0,"13,863.0",550.3
3.0,810.9,385.7
4.0,"2,011.0",8.9
5.0,"7,328.1",18.8
7.0,7.1,nan
8.0,"14,084.3",681.1
9.0,"14,477.0","1,608.7"
10.0,"13,556.8",689.1


## number paths/OD pairs

In [28]:
print('number of od pairs within budget per POI - PuT/PrT')
POI_pairs = trips[trips.Czas_PrT<BUDGET_PRT].groupby(by=['POI'])['odPair'].nunique().to_frame('nODPairsPrT')
POI_pairs['nODPairsPuT'] = trips[trips.Czas_PuT<BUDGET_PUT].groupby(by=['POI'])['odPair'].nunique()
POI_pairs

number of od pairs within budget per POI - PuT/PrT


,nODPairsPrT,nODPairsPuT
POI,,
1.0,156,9.0
2.0,463,17.0
3.0,25,18.0
4.0,106,3.0
5.0,178,6.0
7.0,1,nan
8.0,473,10.0
9.0,528,43.0
10.0,449,14.0


## mean/median time available

In [31]:
print("time available PrT")
trips[trips.Czas_PrT<BUDGET_PRT].groupby(
    by=['POI'])['Czas_PrT', 'time_left_PrT'].agg(['sum','mean','median'])

time available PrT


Czas_PrT             time_left_PrT            
          sum mean median           sum mean median
POI                                                
1.0   1,659.6 10.6   10.6         195.8  1.3    1.2
2.0   4,261.2  9.2    9.5       1,245.6  2.7    2.4
3.0     270.0 10.8   11.1          27.3  1.1    0.8
4.0   1,078.9 10.2   10.5         181.8  1.7    1.4
5.0   1,788.0 10.0   10.5         329.1  1.8    1.4
7.0      11.5 11.5   11.5           0.4  0.4    0.4
8.0   4,552.2  9.6    9.9       1,073.5  2.3    2.0
9.0   4,543.7  8.6    8.8       1,736.1  3.3    3.0
10.0  4,181.9  9.3    9.6       1,158.3  2.6    2.3
11.0  4,004.8  9.7   10.0         919.1  2.2    1.9
12.0  3,153.1  9.8   10.2         664.8  2.1    1.7
13.0  4,509.2  8.4    8.4       1,889.6  3.5    3.5
14.0  4,741.0  9.4    9.5       1,277.2  2.5    2.4
15.0  4,814.6  9.3    9.5       1,358.2  2.6    2.4
16.0  2,290.8 10.4   10.7         325.8  1.5    1.2
17.0  4,404.8  8.2    8.3       1,958.3  3.7    3.6
18.0  4,121.6  8.9    9.1       1,409.0  3.0    2.8
19.0  3,989.0  8.9    9.2       1,351.2  3.0    2.7
20.0  3,282.2  9.3    9.6         916.2  2.6    2.3
21.0  2,060.1 10.0   10.4         390.0  1.9    1.5
22.0  1,937.0 10.0   10.3         370.3  1.9    1.6
23.0  2,060.1  9.9   10.3         425.7  2.0    1.6
24.0    939.5 10.3   10.8         142.9  1.6    1.1
25.0  1,797.5 10.0   10.4         343.4  1.9    1.5
26.0  1,477.9  9.7   10.1         330.0  2.2    1.8
27.0  2,248.1  9.5    9.9         558.8  2.4    2.0
28.0  1,538.8 10.0   10.3         292.9  1.9    1.6
29.0  1,232.7 10.1   10.5         218.3  1.8    1.4
30.0  2,589.9  9.6   10.0         633.3  2.3    1.9
31.0  3,103.2  9.3    9.6         881.2  2.6    2.3
32.0  3,507.2  9.6    9.9         845.9  2.3    2.0

In [33]:
print("time available PuT")
trips[trips.Czas_PuT<BUDGET_PUT].groupby(
    by=['POI'])['Czas_PuT', 'time_left_PuT'].agg(['sum','mean','median'])

time available PuT


Czas_PuT             time_left_PuT            
          sum mean median           sum mean median
POI                                                
1.0      94.3 10.5   11.0          12.8  1.4    0.9
2.0     168.1  9.9    9.9          34.1  2.0    2.0
3.0     177.7  9.9    9.4          36.4  2.0    2.5
4.0      30.9 10.3   11.1           4.8  1.6    0.8
5.0      62.0 10.3   10.9           9.4  1.6    1.0
8.0     101.3 10.1   10.7          17.7  1.8    1.2
9.0     406.3  9.4   10.0         105.2  2.4    1.9
10.0    140.2 10.0   10.3          26.3  1.9    1.6
11.0    339.8 10.0   10.3          64.6  1.9    1.6
12.0    382.2  9.8   10.4          81.6  2.1    1.5
13.0    687.1  7.3    7.2         430.9  4.6    4.7
14.0    376.0  9.6   10.2          87.8  2.3    1.7
15.0    339.8 10.0   10.3          64.6  1.9    1.6
16.0    267.9  9.6   10.0          65.1  2.3    1.9
17.0    625.1  8.2    8.5         278.9  3.7    3.4
18.0    213.1  8.9    9.1          72.3  3.0    2.8
19.0    279.9  7.8    7.9         148.3  4.1    4.0
20.0      9.9  9.9    9.9           2.0  2.0    2.0
21.0    184.0  9.7   10.4          41.9  2.2    1.5
22.0    104.6  9.5   10.4          26.2  2.4    1.5
23.0    157.3 10.5   10.7          21.1  1.4    1.2
24.0     10.0 10.0   10.0           1.9  1.9    1.9
26.0     30.4 10.1   11.2           5.3  1.8    0.7
27.0     55.5  9.3    9.3          15.8  2.6    2.6
28.0    148.1  9.3    9.7          42.2  2.6    2.2
30.0    518.8  9.3    9.5         147.3  2.6    2.4
31.0    380.3 10.0   10.1          71.7  1.9    1.8
32.0    623.5  9.0    9.3         197.2  2.9    2.6